In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from unidecode import unidecode


2023-04-16 03:59:39.433450: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 03:59:40.325074: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 03:59:40.326845: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 03:59:42.003133: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Carregue os dados do CSV em um DataFrame do Pandas:
df = pd.read_csv('./dados.csv', delimiter=';')

# Fazer amostragem aleatória de 20.000 linhas e remover linhas com valores ausentes:
df = df.sample(n=35000).dropna()

# Divida os dados em recursos e rótulos:
X = df['Name']
y = df['Gender']

# Remova acentos e caracteres problemáticos:

def remove_accents(text):
    return unidecode(text)

In [3]:
# Transforme os dados de texto em vetores numéricos usando o CountVectorizer do Scikit-learn:
vectorizer = CountVectorizer(preprocessor=remove_accents)
X = vectorizer.fit_transform(X)

# Divida os dados em conjuntos de treinamento e teste:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Padronize os recursos usando o StandardScaler do Scikit-learn:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.toarray())
X_test = scaler.transform(X_test.toarray())

# Crie um modelo sequencial com o Keras:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile o modelo com a função de perda, otimizador e métricas:
optimizer = Adam(lr=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

2023-04-16 04:00:13.389935: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-16 04:00:13.391589: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/afonsolelis/.local/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [4]:
# Treine o modelo com os dados de treinamento:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train, y_train, epochs=20, batch_size=32,
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/20


2023-04-16 04:00:20.139495: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3832428432 exceeds 10% of free system memory.


875/875 [==============================] - 8s 9ms/step - loss: 0.7645 - accuracy: 0.5584 - val_loss: 0.6940 - val_accuracy: 0.5988
Epoch 2/20
875/875 [==============================] - 7s 7ms/step - loss: 0.3718 - accuracy: 0.8308 - val_loss: 0.9255 - val_accuracy: 0.3893
Epoch 3/20
875/875 [==============================] - 6s 7ms/step - loss: 0.2190 - accuracy: 0.9036 - val_loss: 0.8882 - val_accuracy: 0.5822
Epoch 4/20
875/875 [==============================] - 6s 7ms/step - loss: 0.0837 - accuracy: 0.9609 - val_loss: 0.9963 - val_accuracy: 0.5817


In [5]:
# Avalie o desempenho do modelo com os dados de teste:
score = model.evaluate(X_test, y_test, batch_size=32)
print("Acurácia:", score[1])

219/219 [==============================] - 1s 5ms/step - loss: 0.9963 - accuracy: 0.5817
Acurácia: 0.5816545486450195


In [7]:
# Classifique o nome "Afonso":
name_to_classify = vectorizer.transform(["Afonso"])
predicted_gender = model.predict(name_to_classify)
predicted_gender = np.round(predicted_gender)
print("O gênero do nome Afonso é:", predicted_gender)

1/1 [==============================] - 0s 21ms/step
O gênero do nome Afonso é: [[0.]]


2023-04-16 04:01:11.726033: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [1,3]
	 [[{{node Placeholder/_0}}]]
